# Tác vụ 4: Giao diện hội thoại – Trò chuyện với LLM Llama 3 và Titan Premier

Trong sổ tay này, bạn sẽ xây dựng một chatbot bằng cách sử dụng Mô hình nền tảng (FM) llama3-8b-instruct và titan-text-premier trong Amazon Bedrock.

Các giao diện hội thoại, chẳng hạn như chatbot và trợ lý ảo, có thể giúp bạn nâng cao trải nghiệm người dùng cho khách hàng. Chatbot sử dụng thuật toán xử lý ngôn ngữ tự nhiên (NLP) và máy học để hiểu và phản hồi các truy vấn của người dùng. Bạn có thể dùng chatbot trong nhiều ứng dụng khác nhau, chẳng hạn như dịch vụ khách hàng, bán hàng và thương mại điện tử, để đưa ra phản hồi nhanh chóng và hiệu quả cho người dùng. Người dùng có thể truy cập chatbot thông qua nhiều kênh, chẳng hạn như trang web, nền tảng mạng xã hội và ứng dụng nhắn tin.

- **Chatbot (Cơ bản)**, chatbot Zero Shot với mô hình FM
- **Chatbot sử dụng câu lệnh**, template(LangChain) – Chatbot có một số ngữ cảnh được cung cấp trong mẫu câu lệnh
- **Chatbot có tính cách**, Chatbot có vai trò cụ thể, Ví dụ: Chuyên gia khai vấn nghề nghiệp và Tương tác với con người
- **Chatbot nhận biết ngữ cảnh**, Chuyển ngữ cảnh thông qua tệp bên ngoài bằng cách tạo phần nhúng.

## Khung LangChain để xây dựng Chatbot với Amazon Bedrock

Trong các giao diện hội thoại như chatbot, việc ghi nhớ các tương tác trước đó có ý nghĩa đặc biệt quan trọng, cả ở cấp độ ngắn hạn và dài hạn.

Khung LangChain cung cấp các thành phần bộ nhớ dưới hai hình thức. Đầu tiên, LangChain cung cấp các tiện ích trợ giúp để quản lý và kiểm soát các tin nhắn trò chuyện trước đó. Các tiện ích này được thiết kế theo dạng mô-đun. Thứ hai, LangChain mang đến những cách dễ dàng để kết hợp các tiện ích này vào chuỗi. Nhờ đó, bạn có thể dễ dàng xác định và tương tác với nhiều loại nội dung trừu tượng để đơn giản hóa việc xây dựng các chatbot mạnh mẽ.

## Xây dựng Chatbot có ngữ cảnh – Các yếu tố chính

Để xây dựng chatbot nhận biết ngữ cảnh, việc đầu tiên cần làm là tạo phần nhúng cho ngữ cảnh. Thông thường, bạn sẽ có một quy trình nhập dữ liệu chạy qua mô hình nhúng của mình và tạo ra phần nhúng. Sau đó, phần nhúng sẽ được lưu trữ trong kho lưu trữ véc-tơ. Trong sổ tay này, bạn sẽ dùng mô hình Phần nhúng Titan cho mục đích trên. Quy trình thứ hai là điều phối yêu cầu của người dùng, tương tác, gọi và trả về kết quả. Quy trình này bao gồm việc điều phối yêu cầu của người dùng, tương tác với các mô hình/thành phần cần thiết để thu thập thông tin, gọi chatbot để tạo thành phản hồi, sau đó trả lời người dùng bằng phản hồi của chatbot.

## Tác vụ 4.1: Thiết lập môi trường

Trong tác vụ này, bạn sẽ thiết lập môi trường của mình.

In [ ]:
#ignore warnings and create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))


In [ ]:
# format instructions into a conversational prompt
from typing import Dict, List

def format_instructions(instructions: List[Dict[str, str]]) -> List[str]:
    """Format instructions where conversation roles must alternate system/user/assistant/user/assistant/..."""
    prompt: List[str] = []
    for instruction in instructions:
        if instruction["role"] == "system":
            prompt.extend(["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n", (instruction["content"]).strip(), " <|eot_id|>"])
        elif instruction["role"] == "user":
            prompt.extend(["<|start_header_id|>user<|end_header_id|>\n", (instruction["content"]).strip(), " <|eot_id|>"])
        else:
            raise ValueError(f"Invalid role: {instruction['role']}. Role must be either 'user' or 'system'.")
    prompt.extend(["<|start_header_id|>assistant<|end_header_id|>\n"])
    return "".join(prompt)

## Tác vụ 4.2: Sử dụng lịch sử trò chuyện trong LangChain để bắt đầu cuộc hội thoại

Trong tác vụ này, bạn sẽ cho phép chatbot truyền tải ngữ cảnh hội thoại qua nhiều hoạt động tương tác với người dùng. Để duy trì các cuộc hội thoại có ý nghĩa và mạch lạc theo thời gian, chatbot cần phải có bộ nhớ hội thoại.

Bạn triển khai các khả năng của bộ nhớ hội thoại bằng cách xây dựng dựa trên lớp InMemoryChatMessageHistory của LangChain. Đối tượng này lưu trữ các cuộc hội thoại giữa người dùng và chatbot, cũng như lịch sử trong tác tử chatbot để có thể tận dụng ngữ cảnh của cuộc hội thoại trước đó.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Lưu ý:** Đầu ra của mô hình không mang tính xác định.

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_aws import ChatBedrock

chat_model=ChatBedrock(
    model_id="meta.llama3-8b-instruct-v1:0" , 
    client=bedrock_client)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer the following questions as best you can."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory()


def get_history():
    return history


chain = prompt | chat_model | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key="chat_history",
)
query="how are you?"
response=wrapped_chain.invoke({"input": query})
# Printing history to see the history being built out. 
print(history)
# For the rest of the conversation, the output will only include response

### Câu hỏi mới

Mô hình đã phản hồi bằng tin nhắn ban đầu. Bây giờ, bạn hãy đặt ra cho mô hình một vài câu hỏi.

In [ ]:
#new questions
instructions = [{"role": "user", "content": "Give me a few tips on how to start a new garden."}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

### Xây dựng dựa trên các câu hỏi

Bây giờ, hãy đặt một câu hỏi mà không nhắc đến từ "vườn" để xem mô hình có thể hiểu được cuộc hội thoại trước đó hay không.

In [ ]:
# build on the questions
instructions = [{"role": "user", "content": "bugs"}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

### Kết thúc cuộc hội thoại này

In [ ]:
# finishing the conversation
instructions = [{"role": "user", "content": "That's all, thank you!"}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

## Tác vụ 4.3: Chatbot sử dụng mẫu câu lệnh (LangChain)

Trong tác vụ này, bạn sẽ dùng PromptTemplate mặc định để xây dựng dữ liệu đầu vào này. LangChain cung cấp một số lớp và hàm giúp việc xây dựng và làm việc với câu lệnh trở nên dễ dàng.

In [ ]:
#  prompt for a conversational agent
def format_prompt(actor:str, input:str):
    formatted_prompt: List[str] = []
    if actor == "system":
        prompt_template="""<|begin_of_text|><|start_header_id|>{actor}<|end_header_id|>\n{input}<|eot_id|>"""
    elif actor == "user":
        prompt_template="""<|start_header_id|>{actor}<|end_header_id|>\n{input}<|eot_id|>"""
    else:
        raise ValueError(f"Invalid role: {actor}. Role must be either 'user' or 'system'.")   
    prompt = PromptTemplate.from_template(prompt_template)     
    formatted_prompt.extend(prompt.format(actor=actor,input=input))
    formatted_prompt.extend(["<|start_header_id|>assistant<|end_header_id|>\n"])
    return "".join(formatted_prompt)

In [ ]:
# chat user experience
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            with self.out:
                print("Thank you , that was a nice chat !!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        response = self.qa.invoke({"input": prompt})
                        result=response['answer']
                    else:
                        instructions = [{"role": "user", "content": prompt}]
                        #result = self.qa.invoke({'input': format_prompt("user",prompt)}) #, 'history':chat_history})
                        result = self.qa.invoke({"input": format_instructions(instructions)})
                except:
                    result = "No answer"
                thinking.value=""
                print(f"AI:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

Tiếp theo, hãy bắt đầu một cuộc trò chuyện.

In [ ]:
# start chat
history = InMemoryChatMessageHistory() #reset chat history
chat = ChatUX(wrapped_chain)
chat.start_chat()

In [ ]:
print(history)

## Tác vụ 4.4: Chatbot có tính cách

Trong tác vụ này, trợ lý Trí tuệ nhân tạo (AI) sẽ đóng vai trò là một chuyên gia khai vấn nghề nghiệp. Bạn có thể thông báo cho chatbot về tính cách (hoặc vai trò) của nó bằng thông báo hệ thống. Tiếp tục tận dụng lớp InMemoryChatMessageHistory để duy trì ngữ cảnh hội thoại.

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", " You will be acting as a career coach. Your goal is to give career advice to users. For questions that are not career related, don't provide advice. Say, I don't know."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory() # reset history

chain = prompt | chat_model | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key="career_chat_history",
)

response=wrapped_chain.invoke({"input": "What are the career options in AI?"})
print(response)

In [ ]:
response=wrapped_chain.invoke({"input": "How to fix my car?"})
print(response)

In [ ]:
print(history)

Bây giờ, hãy đặt một câu hỏi không nằm trong chuyên môn của tính cách này. Theo đúng kịch bản thì mô hình sẽ không trả lời được câu hỏi đó và đưa ra lý do.

## Tác vụ 4.5 Chatbot có ngữ cảnh

Trong tác vụ này, bạn sẽ yêu cầu chatbot trả lời các câu hỏi dựa trên ngữ cảnh được đưa vào. Bạn lấy một tệp CSV và dùng mô hình Phần nhúng Titan để tạo véc-tơ biểu thị ngữ cảnh đó. Véc-tơ này được lưu trữ trong Facebook AI Similarity Search (FAISS). Khi một câu hỏi được đặt ra cho chatbot, bạn sẽ chuyển véc-tơ này trở lại chatbot và chatbot sẽ truy xuất câu trả lời bằng véc-tơ đó.

### Mô hình phần nhúng Titan

Phần nhúng biểu diễn các từ, cụm từ hoặc bất kỳ mục rời rạc nào khác dưới dạng véc-tơ trong không gian véc-tơ liên tục. Dựa vào đó, mô hình máy học sẽ thực hiện các phép toán trên những dạng biểu diễn này và nắm bắt mối quan hệ ngữ nghĩa giữa chúng.

Bạn sẽ dùng phần nhúng cho Tạo tăng cường truy xuất (RAG) [khả năng tìm kiếm tài liệu](https://labelbox.com/blog/how-vector-similarity-search-works/).

In [ ]:
# model configuration
from langchain_aws.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate

br_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=bedrock_client)

### FAISS dưới dạng VectorStore

Để sử dụng phần nhúng cho mục đích tìm kiếm, bạn cần một kho lưu trữ có thể tìm kiếm các véc-tơ tương tự một cách hiệu quả. Trong sổ tay này, bạn sẽ sử dụng kho lưu trữ trong bộ nhớ FAISS. Để lưu trữ vĩnh viễn véc-tơ, bạn có thể sử dụng Cơ sở kiến thức cho Amazon Bedrock, pgVector, Pinecone, Weaviate hoặc Chroma.

Các API LangChain VectorStore có sẵn [tại đây] (https://python.langchain.com/v0.2/docs/integrations/vectorstores/).

In [ ]:
# vector store
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

loader = CSVLoader("../rag_data/Amazon_SageMaker_FAQs.csv") # --- > 219 docs with 400 chars
documents_aws = loader.load() #
print(f"documents:loaded:size={len(documents_aws)}")

docs = CharacterTextSplitter(chunk_size=2000, chunk_overlap=400, separator=",").split_documents(documents_aws)

print(f"Documents:after split and chunking size={len(docs)}")
vectorstore_faiss_aws = None
try:
    
    vectorstore_faiss_aws = FAISS.from_documents(
        documents=docs,
        embedding = br_embeddings, 
        #**k_args
    )

    print(f"vectorstore_faiss_aws:created={vectorstore_faiss_aws}::")

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

### Chạy kiểm thử nhanh không cần viết nhiều mã 

Bạn có thể sử dụng lớp Wrapper do LangChain cung cấp để truy vấn kho cơ sở dữ liệu véc-tơ và trả về các tài liệu có liên quan. Thao tác này sẽ chạy Chuỗi QA với tất cả các giá trị mặc định.

In [ ]:
chat_llm=ChatBedrock(
    model_id="amazon.titan-text-premier-v1:0" , 
    client=bedrock_client)
# wrapper store faiss
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)
print(wrapper_store_faiss.query("R in SageMaker", llm=chat_llm))

### Ứng dụng Chatbot

Đối với chatbot, bạn cần quản lý ngữ cảnh, lịch sử, kho lưu trữ véc-tơ và nhiều thành phần khác. Bạn bắt đầu bằng cách xây dựng chuỗi Tạo tăng cường truy xuất (RAG) hỗ trợ ngữ cảnh.

Quá trình này sử dụng các hàm **create_stuff_documents_chain** và **create_retrieval_chain**

### Các tham số và hàm được sử dụng cho RAG

- **Retriever:** Bạn đã sử dụng `VectorStoreRetriever` do một `VectorStore` hỗ trợ. Để truy xuất văn bản, bạn có thể chọn một trong hai kiểu tìm kiếm: `"similarity"` hoặc `"mmr"`. `search_type="similarity"` sử dụng chức năng tìm kiếm tương tự trong đối tượng truy xuất, trong đó các véc-tơ đoạn văn bản giống véc-tơ câu hỏi nhất sẽ được chọn.

- **create_stuff_documents_chain** chỉ định cách ngữ cảnh truy xuất được đưa vào câu lệnh và LLM. Tài liệu truy xuất sẽ được "nhồi" vào câu lệnh dưới dạng ngữ cảnh mà không tóm tắt hay xử lý bất kỳ nội dung gì.

- **create_retrieval_chain** sẽ thêm bước truy xuất và truyền ngữ cảnh đã truy xuất thông qua chuỗi để cung cấp ngữ cảnh đó cùng với câu trả lời cuối cùng. 

Nếu câu hỏi đặt ra nằm ngoài phạm vi của ngữ cảnh thì mô hình sẽ trả lời là mô hình không biết câu trả lời.

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

retriever=vectorstore_faiss_aws.as_retriever()
question_answer_chain = create_stuff_documents_chain(chat_llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "What is sagemaker?"})
print(response) # shows the document chunks consulted to come up with the answer

Tiếp theo, hãy bắt đầu một cuộc trò chuyện

In [ ]:
chat = ChatUX(rag_chain, retrievalChain=True)
chat.start_chat()  # Only answers will be shown here, and not the citations


Bạn đã sử dụng LLM Titan để tạo giao diện hội thoại với các mẫu hình sau:

- Chatbot (Cơ bản – không có ngữ cảnh)
- Chatbot sử dụng mẫu câu lệnh (Langchain)
- Chatbot có tính cách
- Chatbot với ngữ cảnh

### Tự thực hiện

- Thay đổi câu lệnh theo trường hợp sử dụng cụ thể của bạn và đánh giá đầu ra của các mô hình khác nhau.
- Thử nghiệm với độ dài mã token để nắm được độ trễ và khả năng phản hồi của dịch vụ.
- Áp dụng các nguyên tắc tạo câu lệnh khác nhau để có được đầu ra tốt hơn.

### Dọn dẹp

Bạn đã hoàn thành sổ tay này. Để chuyển sang phần tiếp theo của phòng thực hành, hãy làm như sau:

- Đóng tệp sổ tay này và tiếp tục với **Tác vụ 5**.